# Dietary Restriction AI

### Import Packages

In [1]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.llms import Ollama
import os
import json
import pandas as pd 
import tempfile 
from pyspark.sql import SparkSession

### Initialize a Spark Session

In [5]:
spark = SparkSession.builder \
    .appName("Local Spark") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/29 11:20:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Data Ingestion

In [6]:
all_files = os.listdir("./")
recipe_files = [file for file in all_files if "recipes_raw_nosource" in file]

df_list = []

for file_name in recipe_files: 
    temp_path = os.path.join(tempfile.gettempdir(), file_name)
    
    with open(file_name, "r", encoding="utf-8") as file:
        data = json.load(file)

    file_df = pd.DataFrame.from_dict(data, orient="index")
    df_list.append(file_df)  # Collect DataFrame
    
# Concatenate all dataframes
df = pd.concat(df_list)
df = df.reset_index().rename(columns={"index": "id"})

# repartition the dataframe 
df_spark = spark.createDataFrame(df)
df_spark = df_spark.repartition(100)

df_spark.show()

25/03/29 11:21:00 WARN TaskSetManager: Stage 0 contains a task of very large size (16937 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|        instructions|         ingredients|               title|        picture_link|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|QN5CYubunXaDelXyw...|In a blender, com...|[1 (14.5-ounce) c...|     Virgin Daiquiri|                NULL|
|hKM.NJ24HSFF1GkQB...|1. Whisk the chil...|[1/2 cup chili po...|Chili BBQ'd Chick...|                NULL|
|rtPpWjYBQBuNXsFtA...|Make batter: In a...|[1 cup plus 2 tab...|Individual Yorksh...|                NULL|
|zm9UAJsjPjmAU6r7J...|Place all the fil...|[1 1/2 cups chopp...|     Lobster Ravioli|                NULL|
|sJWOvBHePd88CbaQG...|Preheat a grill t...|[3 ears corn, 1 s...|Avocado and Grill...|n9xn6z4PPJboIDlB0...|
|R3TxL4huVQliNiDW4...|Preheat oven to 4...|[3 tablespoons ca...|Spinach and Potat...|                NULL|
|ixEoBXuptWDGZo.gB...|Mix the BBQ Pul

### Model Ingestion

In [7]:
df_spark.unpersist()

DataFrame[id: string, instructions: string, ingredients: array<string>, title: string, picture_link: string]

In [8]:
llm = Ollama(model="llama3.2")
print("Loaded Model")

Loaded Model


/var/folders/2l/vxt6rlxn46s5z8qm6j10lrg00000gp/T/ipykernel_6927/4125328510.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [9]:
recipe = """ 
Ingredients
1 ½ cups all-purpose flour, 3 ½ teaspoons baking powder, 1 tablespoon white sugar, ¼ teaspoon salt, or more to taste, 
1 ¼ cups milk, 3 tablespoons butter, melted, 1 large egg

Directions
Sift flour, baking powder, sugar, and salt together in a large bowl. Make a well in the center and add milk, melted butter, and egg; mix until smooth.

Heat a lightly oiled griddle or pan over medium-high heat. Pour or scoop the batter onto the griddle, using approximately 1/4 cup for each pancake; cook until bubbles form and the edges are dry, about 2 to 3 minutes. Flip and cook until browned on the other side. Repeat with remaining batter."

"""
prompt_append = " Modify this recipe so that it is more suited for a high-protein, low carb diet. Provide a list of macronutrients as a part of the analysis." 
prompt = recipe + prompt_append

response = llm.invoke(prompt)
print(response)

To modify the pancake recipe to suit a high-protein, low-carb diet, we'll make some key changes:

1. Replace all-purpose flour with an almond flour-based mixture: Almond flour is a good source of protein and has a lower carb content compared to traditional flour.
2. Use protein-rich milk alternative: We'll replace regular milk with unsweetened almond milk or another low-carb milk alternative, such as coconut milk.
3. Add more protein-rich ingredients: Introduce some protein powder (e.g., whey or pea) to boost the protein content of each pancake.

Here's the modified recipe:

Ingredients:

* 1 ½ cups almond flour
* 3 ½ teaspoons baking powder (make sure it's sugar-free)
* 1 tablespoon coconut sugar (or another low-carb sweetener)
* ¼ teaspoon salt
* 1/4 cup protein powder (whey or pea-based)
* 1 ¼ cups unsweetened almond milk
* 3 tablespoons melted butter
* 1 large egg

Directions:

1. Sift the almond flour, baking powder, coconut sugar, and salt together in a large bowl.
2. Make a well